In [28]:
import stanza
import requests
import pandas as pd

# Načítanie textového súboru
def read_text_file(filepath):
    with open(filepath, "r", encoding="utf-8") as file:
        return file.read()

# Funkcia na posielanie požiadavky na MorphoDiTa API
def tag_with_morphodita(sentence):
    api_url = "https://lindat.mff.cuni.cz/services/morphodita/api/tag?"
    params = {
        "data": sentence,
        "model": "slovak-morfflex-pdt-170914-no_dia",  # Definovanie modelu priamo tu
        "guesser": "yes",  # Použiť morfologický hádač
        "input": "untokenized",  # Vstup ako obyčajný text
        "convert_tagset": "strip_lemma_id",  # Upraviť tag set
        "output": "json"  # JSON formát výstupu
    }
    response = requests.get        (api_url, params=params)
    print(response)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Chyba API: {response.status_code} - {response.text}")

# Cesta k textovému súboru
filepath = r"C:\Users\marti\Downloads\DiploDiktaty.txt"
text = read_text_file(filepath)

# Inicializácia Stanza pre tokenizáciu
stanza.download('sk')
nlp = stanza.Pipeline(lang='sk', processors='tokenize')

# Tokenizácia textu na vety
document = nlp(text)

# Príprava dát na uloženie
sentences_data = []  # Pre CSV s vetami
words_data = []      # Pre CSV so slovami

# Prechádzanie cez tokenizované vety
for sentence_id, sentence in enumerate(document.sentences, start=1):
    sentences_data.append([sentence_id, sentence.text])  # Uloženie vety s ID
    try:
        # Tagovanie cez MorphoDiTa API
        morphodita_result = tag_with_morphodita(sentence.text)

        # Spracovanie výsledkov API
        if "result" in morphodita_result and isinstance(morphodita_result["result"], list):
            for token in morphodita_result["result"]:
                words_data.append([
                    sentence_id,             # ID vety
                    token.get("token", ""),  # Token (slovo)
                    token.get("lemma", ""),  # Lemma
                    token.get("tag", "")     # Morfologický tag
                ])
        else:
            print(f"Odpoveď API nemá správny formát pre vetu: {sentence.text}")
    except Exception as e:
        print(f"Chyba pri spracovaní vety '{sentence.text}': {e}")

# Uloženie výsledkov do CSV
sentences_df = pd.DataFrame(sentences_data, columns=["Sentence_ID", "Text"])
words_df = pd.DataFrame(words_data, columns=["Sentence_ID", "Token", "Lemma", "Tag"])

sentences_df.to_csv("sentences.csv", index=False, encoding="utf-8")
words_df.to_csv("words.csv", index=False, encoding="utf-8")

print("Výstup uložený do 'sentences.csv' a 'words.csv'.")


AttributeError: 'ufal.morphodita.Tagger' object has no attribute 'setLemmaAlphabet'

In [ ]:
filepath = r"C:\Users\marti\Downloads\DiploDiktaty.txt"